In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import numpy as np
import sys
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
import scipy.io as io
import os
import random

# I Data

## 1. Convert Signals to 2-D Images

In [ ]:
'''Convert signals to 2-D images'''

train = []
train_label = []
test = []
test_label = []

for d_type in range(10):                         # 10类 Data, 每一类下有4份独立数据(rpm)
    data = Data[d_type]                          # 取第(d_type)类, len(data)=4
    for rpm in range(4):
        data_load = data[rpm]                    # 选取其中一个speed
        max_start = len(data_load) - 4096        # 每4096(64*64)个采样点组成一个样本，最后一个样本的初始采样点为max_start
        starts = []

        # Train
        for i in range(700):
            # 随机采样，即随机选取一个start, 该start不在starts[]里,就加入
            while True:
                start = random.randint(0, max_start)
                if start not in starts:
                    starts.append(start)
                    break

            # 以start为始，顺取4096个采样点，堆叠为二维数据
            img = data_load[start : start + 4096]
            img = np.array(img)

            # 生成一个训练样本，对应生成一个label
            train.append(img.reshape(64, 64))   
            train_label.append(d_type)
            
        # Test
        for i in range(300):
            while True:
                start = random.randint(0, max_start)
                if start not in starts:
                    starts.append(start)
                    break

            img = data_load[start : start + 4096]
            img = np.array(img)
            test.append(img.reshape(64, 64))
            test_label.append(d_type)
    
print(len(train))
print("train:%d"%(len(train)))
print("train_label:%d"%(len(train_label)))
print("test:%d"%(len(test)))
print("test_label:%d"%(len(test_label)))

# 最终训练样本：28000 = 700*4*10   测试样本：12000 = 300*4*10

np.savez(root + "data_img/train", *train)
np.savez(root + "data_img/train_label", *train_label)
np.savez(root + "data_img/test", *test)
np.savez(root + "data_img/test_label", *test_label)

28000
train:28000
train_label:28000
test:12000
test_label:12000


## 2. Dataset Definition

In [ ]:
class MyData(Dataset):
    def __init__(self, img, label):
        self.img = img
        self.label = label

    def __getitem__(self, index):
        assert index < len(self.img)
        return torch.Tensor([self.img[index]]), self.label[index]

    def __len__(self):
        return len(self.img)

    def get_tensor(self):
        return torch.Tensor([self.img]), torch.Tensor(self.label)

## 3. TrainLoader / TestLoader

In [ ]:
num_epoch = 100
batch_size = 8
lr = 0.001

In [ ]:
root = '/content/drive/MyDrive/Colab_Notebooks/CWRU/data_2/'

train_img_dict = np.load(root + "data_img/train.npz")
train_label_dict = np.load(root + "data_img/train_label.npz")

train_img = []
train_label = []

for i in train_img_dict.files:
    train_img.append(train_img_dict[i])
    train_label.append(train_label_dict[i])

# print(len(train_img))      28000
# print(len(train_label))    28000

trainset = MyData(train_img, train_label)
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          num_workers = 2
                                        )
for X, y in trainloader:
    print(X.shape)  # torch.Size([8, 1, 64, 64])
    print(y.shape)  # torch.Size([8])

In [ ]:
test_img_dict = np.load(root + "data_img/test.npz")
test_label_dict = np.load(root + "data_img/test_label.npz")

test_img = []
test_label = []

for i in test_img_dict.files:
    test_img.append(test_img_dict[i])
    test_label.append(test_label_dict[i])

testset = MyData(test_img, test_label)
testloader = torch.utils.data.DataLoader(testset,
                                         batch_size = batch_size,
                                         shuffle = True,
                                         num_workers = 2
                                         )

for X, y in testloader:
    print(X.shape)  # torch.Size([8, 1, 64, 64])
    print(y.shape)  # torch.Size([8])

# II Model

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 5, padding=2), # in_channels, out_channels, kernel_size
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # kernel_size, stride
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc = nn.Sequential(
            nn.Linear(256*4*4, 2560),
            nn.ReLU(),
            nn.Linear(2560, 1000),
            nn.ReLU(),
            nn.Linear(1000, 10)
        )

    def forward(self, img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))
        return output

In [ ]:
net = Net()
print(net)

Net(
  (conv): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=4096, out_features=2560, bias=True)
    (1): ReLU()
    (2): Linear(in_features=2560, out_features=1000, bias=True)
    (3): ReLU()
    (4): Linear(in_features=1000, out_features=10, bias=True)
  )
)


# III Train_Model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

net = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr)

for epoch in range(num_epoch):
    run_loss = 0
    for i, data in enumerate(trainloader):
        img, label = data
        img = img.cuda()
        label = label.cuda()
        output = net(img)

        loss = criterion(output, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        run_loss += loss
        
        if i % 1000 == 999:
            print("epoch:", epoch, "[", i-999, ":", i, "]", "loss:", run_loss.item()/1000)
            run_loss = 0
        

    path  = "/content/drive/MyDrive/Colab_Notebooks/CWRU/cnn_net.pth"
    torch.save(net.state_dict(), path)
    print("save successfully!")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  


epoch: 0 [ 0 : 999 ] loss: 0.5723403930664063
epoch: 0 [ 1000 : 1999 ] loss: 0.10995864105224609
epoch: 0 [ 2000 : 2999 ] loss: 0.06821983337402343
save successfully!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  


epoch: 1 [ 0 : 999 ] loss: 0.07652546691894531
epoch: 1 [ 1000 : 1999 ] loss: 0.0549166259765625
epoch: 1 [ 2000 : 2999 ] loss: 0.023741888046264648
save successfully!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  


...

epoch: 97 [ 0 : 999 ] loss: 1.3483850052580238e-06
epoch: 97 [ 1000 : 1999 ] loss: 1.6445480287075042e-07
epoch: 97 [ 2000 : 2999 ] loss: 7.672433275729418e-08
save successfully!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  


epoch: 98 [ 0 : 999 ] loss: 6.735101487720385e-08
epoch: 98 [ 1000 : 1999 ] loss: 1.6935280291363596e-07
epoch: 98 [ 2000 : 2999 ] loss: 2.2738884581485765e-08
save successfully!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  


epoch: 99 [ 0 : 999 ] loss: 3.1290819606510925e-08
epoch: 99 [ 1000 : 1999 ] loss: 1.5571557014482096e-08
epoch: 99 [ 2000 : 2999 ] loss: 5.051467724115355e-09
save successfully!


# IV Test_Model

In [ ]:
path = "/content/drive/MyDrive/Colab_Notebooks/CWRU/cnn_net.pth"
net = Net().to(device)
net.load_state_dict(torch.load(path))
print("Load sucessfully!")

correct = 0
total = 0
with torch.no_grad():
    for img, label in testloader:
        img = img.cuda()
        label = label.cuda()
        output = net(img)
        _, pred = torch.max(output, 1)
        total += batch_size
        correct += (pred == label).sum().item()

print("correct:", float(correct / total) * 100, "%")

Load sucessfully!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  


correct: 99.98333333333333 %
